In [1]:
import ase
from ase import Atom, Atoms
from ase.io import read,write
from pyace import PyACECalculator
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import json
from ase.build import bulk, molecule
import matplotlib as mpl
plt.style.use('~/plot.mplstyle')
from matplotlib import pyplot
from ase.eos import EquationOfState

from ase.filters import UnitCellFilter
from ase.optimize import BFGS
from ase.filters import Filter
from ase.eos import calculate_eos
import copy

### Make xyz

In [5]:
atoms=read('../../data/paper/crystalline/POSCAR_bulk')
del atoms[-1]
write('../../data/paper/crystalline/POSCAR-v_Si', atoms, format='vasp')

In [6]:
atoms=read('../../data/paper/crystalline/POSCAR_bulk')
atoms[-1].number=1
write('../../data/paper/crystalline/POSCAR-H_Si', atoms, format='vasp')

In [7]:
atoms=read('../../data/paper/crystalline/POSCAR_bulk')

atoms.append(Atom('H', position=atoms.positions[1]/2))
write('../../data/paper/crystalline/POSCAR-H_i', atoms, format='vasp')

In [8]:
atoms=molecule('H2')
atoms.set_cell([20,20,20])
write('../../data/paper/crystalline/POSCAR_H2', atoms, format='vasp')

In [9]:
si_supercell=read('../../data/paper/crystalline/POSCAR_bulk')  # lattice constant ~5.43 Å

cell = si_supercell.get_cell()
tetra_site = 0.25 * (cell[0] + cell[1] + cell[2])

h2_direction = np.array([1, 1, 0])
h2_direction = h2_direction / np.linalg.norm(h2_direction)

h_bond_length = 0.74

h1_pos = tetra_site - 0.5 * h_bond_length * h2_direction
h2_pos = tetra_site + 0.5 * h_bond_length * h2_direction

h2 = Atoms('H2', positions=[h1_pos, h2_pos])
si_with_h2 = si_supercell + h2


write('../../data/paper/crystalline/POSCAR_wH2', si_with_h2, format='vasp')


### DFT data

In [2]:
# isolated atom energies
dft_ref_e={'Si':-4.14594262, 'H':0.10877632} 

In [3]:
mol=read('../../data/paper/crystalline/DFT/H2/OUTCAR', format='vasp-out')
mol_en=mol.get_potential_energy()/len(mol)
# mol_en=-0.691210334540/2

In [5]:
mol_en*2

-6.91212539

In [13]:
bulk=read('../../data/paper/crystalline/DFT/dia/bulk/OUTCAR')
print(bulk.cell[0]/2)
bulk_en=bulk.get_potential_energy()
bulk_per_atom=bulk_en/len(bulk)
print(bulk_per_atom-dft_ref_e['Si'])

[5.43 0.   0.  ]
-5.86354920109375


In [14]:
## Bulk modulus
with open('../../data/paper/crystalline/DFT/dia/dia_EV_dict.json', 'r') as openfile:
    json_object = json.load(openfile)

dft_vol=np.asarray(json_object['volumes'])
dft_en=np.asarray(json_object['energies'])
eos = EquationOfState(dft_vol, dft_en, eos='murnaghan')
v0, e0, B = eos.fit()
print(B)
print(B*160.2176621) # GPa

0.5877591797607182
94.16940165907592


In [15]:
## Si vacancy
# unrelaxed
atoms=read('../../data/paper/crystalline/DFT/dia/Si_vacancy/unrelaxed/OUTCAR')
atoms_en=atoms.get_potential_energy()

print(atoms_en-(len(atoms)-1)*bulk_en/len(atoms))


4.051862236349166


In [17]:
## Si vacancy
# relaxed
atoms=read('../../data/paper/crystalline/DFT/dia/Si_vacancy/relaxed/OUTCAR')
atoms_en=atoms.get_potential_energy()

print(atoms_en-(len(atoms)-1)*bulk_en/len(atoms))

3.941332176349192


In [18]:
## H interstitial
# unrelaxed
atoms=read('../../data/paper/crystalline/DFT/dia/H_interstitial/unrelaxed/OUTCAR')
atoms_en=atoms.get_potential_energy()
print(atoms_en)

print(atoms_en-bulk_en-mol_en)

-638.41760499
5.645934255000029


In [19]:
## H interstitial
# relaxed
atoms=read('../../data/paper/crystalline/DFT/dia/H_interstitial/relaxed/OUTCAR')
atoms_en=atoms.get_potential_energy()

print(atoms_en-bulk_en-mol_en)

1.564160415000046


In [21]:
## H2 interstitial
# unrelaxed
atoms=read('../../data/paper/crystalline/DFT/dia/H2_interstitial/unrelaxed/OUTCAR')
atoms_en=atoms.get_potential_energy()
print(atoms_en)
v_h2_en=atoms_en-bulk_en-mol_en*2
print(v_h2_en/2)

-646.19262145
0.6634902449999771


In [ ]:
## H2 interstitial
# relaxed
atoms=read('../../data/paper/crystalline/DFT/dia/H2_interstitial/relaxed/OUTCAR')
atoms_en=atoms.get_potential_energy()
print(atoms_en)

print(atoms_en-bulk_en-mol_en*2)

-646.25371724
1.2658846999999867


### ACE potential

In [6]:
potfilename='../../models/SiH-ACE-25.yaml'
acepot = PyACECalculator(potfilename)

In [4]:
bulk_si=read('../../data/paper/crystalline/DFT/dia/bulk/POSCAR')
bulk_si.calc=acepot
ucf = UnitCellFilter(bulk_si, hydrostatic_strain=True)

opt = BFGS(ucf)
opt.run(fmax=0.0001)

# 5. Output final results
print("Final energy per atom: {:.5f} eV".format(bulk_si.get_potential_energy() / len(bulk_si)))
print(f"Relaxed lattice constant: {bulk_si.cell.lengths()[0]/2}")
bulk_en=bulk_si.get_potential_energy()
bulk_per_atom=bulk_en/len(bulk_si)

write('../../data/paper/crystalline/bulk_ace_relaxed.xyz', bulk_si)

      Step     Time          Energy          fmax
BFGS:    0 12:44:22     -374.858879        0.099351
BFGS:    1 12:44:22     -374.859300        0.098567
BFGS:    2 12:44:22     -374.885354        0.001680
BFGS:    3 12:44:23     -374.885362        0.000028
Final energy per atom: -5.85758 eV
Relaxed lattice constant: 5.41500999953023


In [5]:
eos = calculate_eos(bulk_si)

v, e, B = eos.fit()
print(B)
print(B*160.2176621)


0.6121930943917208
98.08414633720614


In [9]:
mol=read('../../data/paper/crystalline/POSCAR_H2')
mol.calc=acepot
mol_en=mol.get_potential_energy()/len(mol)
mol_en*2

-5.933692504500922

In [29]:
## Si vacancy
# unrelaxed
v_si=read('../../data/paper/crystalline/bulk_ace_relaxed.xyz')
del v_si[-1]
v_si.calc=acepot
v_si_en=v_si.get_potential_energy()
write('../../data/paper/crystalline/v_si_unrelaxed.xyz', v_si)

print(v_si_en-(len(v_si)-1)*bulk_en/len(v_si))

3.8660852130555554


In [30]:
## Si vacancy
# relaxed

dyn = BFGS(v_si)
dyn.run(fmax=0.0001)
write('../../data/paper/crystalline/v_si_relaxed.xyz', v_si)

v_si_r_en=v_si.get_potential_energy()
print(v_si_r_en-(len(v_si)-1)*bulk_en/len(v_si))

      Step     Time          Energy          fmax
BFGS:    0 16:04:16     -365.068715        0.563111
BFGS:    1 16:04:16     -365.106791        0.564951
BFGS:    2 16:04:16     -365.319283        0.485838
BFGS:    3 16:04:16     -365.355196        0.427856
BFGS:    4 16:04:16     -365.488698        0.156818
BFGS:    5 16:04:16     -365.498935        0.118060
BFGS:    6 16:04:16     -365.512856        0.070553
BFGS:    7 16:04:16     -365.516423        0.043614
BFGS:    8 16:04:16     -365.518292        0.046405
BFGS:    9 16:04:16     -365.519545        0.039372
BFGS:   10 16:04:17     -365.520659        0.021591
BFGS:   11 16:04:17     -365.521106        0.015158
BFGS:   12 16:04:17     -365.521272        0.013506
BFGS:   13 16:04:17     -365.521361        0.011245
BFGS:   14 16:04:17     -365.521415        0.007215
BFGS:   15 16:04:17     -365.521438        0.004883
BFGS:   16 16:04:17     -365.521451        0.005039
BFGS:   17 16:04:17     -365.521463        0.003702
BFGS:   18 16:

In [ ]:

atoms1 = read('POSCAR_v_Si')
atoms2 = read('../../data/paper/crystalline/v_si_relaxed.xyz')

write('ace_relaxation.xyz', [atoms1, atoms2])


In [31]:
## H interstitial
# unrelaxed
h_i=read('../../data/paper/crystalline/bulk_ace_relaxed.xyz')
h_i.append(Atom('H', position=h_i.positions[1]/2))

h_i.calc=acepot
h_i_en=h_i.get_potential_energy()


# print(h_i_en-bulk_en-mol_en)
# print(h_i.positions)

In [32]:
## H interstitial
# relaxed
dyn = BFGS(h_i)
dyn.run(fmax=0.0001)
write('../../data/paper/crystalline/h_i_relaxed.xyz', h_i)
h_i_r_en=h_i.get_potential_energy()

print(h_i_r_en-bulk_en-mol_en)

      Step     Time          Energy          fmax
BFGS:    0 22:05:43     -373.245275       12.829073
BFGS:    1 22:05:43     -376.227139        4.521013
BFGS:    2 22:05:43     -376.920609        2.029340
BFGS:    3 22:05:43     -377.195285        0.452274
BFGS:    4 22:05:43     -377.241240        0.328257
BFGS:    5 22:05:43     -377.263835        0.276224
BFGS:    6 22:05:43     -377.280915        0.263587
BFGS:    7 22:05:43     -377.291009        0.115024
BFGS:    8 22:05:43     -377.294808        0.081673
BFGS:    9 22:05:43     -377.296545        0.077082
BFGS:   10 22:05:43     -377.298124        0.083082
BFGS:   11 22:05:44     -377.299230        0.041809
BFGS:   12 22:05:44     -377.299841        0.024417
BFGS:   13 22:05:44     -377.300271        0.033349
BFGS:   14 22:05:44     -377.300690        0.042229
BFGS:   15 22:05:44     -377.301025        0.027020
BFGS:   16 22:05:44     -377.301219        0.015260
BFGS:   17 22:05:44     -377.301335        0.013258
BFGS:   18 22:

In [33]:
## H2 interstitial
# unrelaxed
# bulk=read('bulk_ace_relaxed.xyz')
# h2 = Atoms('H2', positions=[h1_pos, h2_pos])
# h2i = bulk + h2

h2i=read('../../data/paper/crystalline/POSCAR_wH2')
h2i.calc=acepot
atoms_en=h2i.get_potential_energy()
print(atoms_en)


print(atoms_en-bulk_en-mol_en*2)

-380.2793465972708
0.5397076345280958


In [34]:
## H2 interstitial
# relaxed
dyn = BFGS(h2i)
dyn.run(fmax=0.0001)
write('../../data/paper/crystalline/h_2i_relaxed.xyz', h2i)
h2i_r_en=h2i.get_potential_energy()

print(h2i_r_en-bulk_en-mol_en*2)

      Step     Time          Energy          fmax
BFGS:    0 22:06:30     -380.279347        1.230819
BFGS:    1 22:06:30     -380.316093        0.337996
BFGS:    2 22:06:30     -380.329206        0.227374
BFGS:    3 22:06:30     -380.343910        0.403404
BFGS:    4 22:06:30     -380.351913        0.303696
BFGS:    5 22:06:30     -380.356911        0.109460
BFGS:    6 22:06:30     -380.358589        0.083856
BFGS:    7 22:06:30     -380.359636        0.098221
BFGS:    8 22:06:30     -380.360092        0.049621
BFGS:    9 22:06:30     -380.360309        0.026940
BFGS:   10 22:06:30     -380.360489        0.044155
BFGS:   11 22:06:31     -380.360639        0.044078
BFGS:   12 22:06:31     -380.360731        0.022627
BFGS:   13 22:06:31     -380.360782        0.011616
BFGS:   14 22:06:31     -380.360819        0.019703
BFGS:   15 22:06:31     -380.360853        0.019499
BFGS:   16 22:06:31     -380.360882        0.009438
BFGS:   17 22:06:31     -380.360906        0.006989
BFGS:   18 22: